# Markov Chain

This exercise is from the Linear Algebra textbook, *Linear Algebra, Gateway to Mathematics*, originally published in 1994. The problem is defined as Exercise 2 of Section 5.4.

## Random Walk

*The random walk on a line is a classical example of an absorbing Markov chain. Suppose a particle moves among the positions 1, 2, 3, 4, and 5 arranged along a line. From positions 2, 3, and 4 the particle moves one position to the left with probability 1/2 and moves on position to the right with probability 1/2. Positions 1 and 5 are absorbing: once the particle reaches either of these positions, it remains in that position forever.*


## Problem

*Write a computer program to simulate the random walk discussed in this section. Have the program repeat the walk a large number of times and print out the average number of visits to the non-absorbing states as well as the average number of steps until absorption. Compare the results of the algorithm to the theoretical results obtained in this section.*

## Simulation

In [7]:
import math
import numpy as np

In [10]:
def create_start_positions(m):
    """
    Create n random start positions. The start positions
    should all be starting from a non-absorbing state.
    """
    start_positions = np.floor(np.random.rand(m) * 3 + 2)
    return start_positions
        

In [15]:
def simulate_walk(start_positions, n):
    """
    Run a simulation of the random walk described above run
    this simulation across all the starting positions given.
    The random walk is run n times.

    Returns a list of numbers, each representing the number of
    steps a particle took before entering an absorted state.
    If a particle, after n passes, was never absorted, then
    the number of steps will be n for that particle.
    """
    
    m = len(start_positions)

    current_positions = np.array([p for p in start_positions])
    step_counts = np.zeros_like(current_positions)
    
    for i in range(n):
        # Don't let particles that are absorbed take a step.
        not_absorbed_mask = ((current_positions != 1) & (current_positions != 5)).astype(float)

        random_step = ((np.random.rand(m) >= 0.5).astype(float) * 2) - 1
        random_step = random_step * not_absorbed_mask
        
        current_positions = current_positions + random_step
        
        increment = np.ones_like(current_positions) * not_absorbed_mask
        step_counts = step_counts + increment
        
    return step_counts
    
    

In [36]:
# Running the simulation
particle_positions = create_start_positions(1000)
step_counts = simulate_walk(particle_positions, n=100000)

np.mean(step_counts)

3.386

## Theoretical

We will use the *Fundamental Matrix of aborbing Markov chains* to calculate the expected number of steps from any non-aborbing state.

In [37]:
# Step 1: Define the Markov Matrix. Note that the aborbing
# states are set to be the first 2 rows / columns of the
# matrix.

markov_matrix = np.array([[1., 0., .5, 0., 0.],
                          [0., 1., 0., 0., .5],
                          [0., 0., 0., .5, 0.],
                          [0., 0., .5, 0., .5],
                          [0., 0., 0., .5, 0.]])

# Step 2: Using the markov matrix, extract the transition
# matrix between non-aborbing steps.

na_transition_matrix = markov_matrix[2:, 2:]

# Step 3: From the non-absorbing transition matrix,
# calculate the fundamental matrix.
fundamental_matrix = np.linalg.inv(np.eye(na_transition_matrix.shape[0]) - na_transition_matrix)

# Step 4: The sum of entries in column i of the fundamental
# matrix contains the expected number of steps before a
# particle starting at position i reaches an absorbing state.
# We are assuming that a particle starts at any position
# with equal probability, so by averaging the sum of the columns,
# we get the overall expected number of steps before a particle
# reaches an absorbing state.
expected_step_count = np.mean(np.sum(fundamental_matrix, axis=0))

expected_step_count

3.3333333333333335